In [1]:
from typing import Any

import pandas as pd


def create_bets_df(bets_data: list[dict[str, Any]]) -> pd.DataFrame:
    data_keys = ['bet_id', 'bet_value', 'user_winnings', 'bet_game_id', 'date']
    bets_df = pd.DataFrame(data=bets_data, columns=data_keys)
    bets_df['date'] = pd.to_datetime(bets_df['date']).dt.date
    return bets_df


def prepare_df_for_bets_graph(
    df: pd.DataFrame,
    game_id: int | None = None
) -> pd.DataFrame:
    df_copy = df.copy(deep=True)
    all_dates = pd.date_range(
        start=df_copy['date'].min(),
        end=df_copy['date'].max(),
        freq='D'
    )
    df_copy['date'] = pd.to_datetime(df_copy['date'])
    if game_id:
        df_copy = df_copy[df_copy['bet_game_id'] == game_id]
    grouped_df = df_copy.groupby('date', as_index=False).agg({
        'bet_id': 'count',
        'bet_value': 'sum',
        'user_winnings': 'sum',
    }).rename(columns={'bet_id': 'bet_count'})
    
    all_dates_df = pd.DataFrame({'date': all_dates})
    grouped_df = all_dates_df.merge(grouped_df, on='date', how='left')
    grouped_df = grouped_df.fillna(0)
    
    grouped_df['profit'] = grouped_df['bet_value'] - grouped_df['user_winnings']
    return grouped_df


In [2]:
GAME_NAMES = [
    'Slots',
    'Dice',
    'Wheel',
    'Roulette',
    'Dice Side',
    'RPS',
    'Keno',
    'Hi/Lo',
    'Limbo',
    'Coinflip'
]


ButtonArgs = list[dict[str, list[bool]]]
Button = dict[str, str | ButtonArgs]


def create_plot_buttons() -> list[Button]:
    buttons_list = [
        {
            'label': 'Total Data',
            'method': 'update',
            'args': [{'visible': [True] * 3 + [False] * 63}]
        },
        {
            'label': 'Add Mean Data',
            'method': 'update',
            'args': [{'visible': [True] * 6 + [False] * 60}]
        }             
    ]
    for i, name in enumerate(GAME_NAMES):
        button_dict = {
            'label': f'{name} Data',
            'method': 'update',
            'args': [
                {
                    'visible': [False] * (6 + i * 6) + [True] * 6 + [False] * (54 - i * 6)
                }
            ]
        }
        buttons_list.append(button_dict)
    return buttons_list


def create_game_hoverstring(game: str, y_label: str) -> str:
    hoverstring = f'Game: {game}<br>{y_label}' + ': %{y}<br>Date: %{x}'
    return hoverstring

In [3]:
import plotly.graph_objects as go


def create_bets_graph(bets_df: pd.DataFrame) -> str:
    layout = go.Layout(
        title='',
        xaxis={'title': None, 'showgrid': False},
        yaxis={'title': None, 'showgrid': False},
        font_color='#e8bcb9',
        paper_bgcolor='#1d1a39',
        plot_bgcolor='#1d1a39',
        showlegend=True,
    )
    fig = go.Figure(layout=layout)
    fig.update_layout(
        legend={
            'yanchor':"top",
            'xanchor':"left",
            'y':-0.59,
            'x':0,
            'bgcolor':'#451952',
        },
    )
    bets_grouped = prepare_df_for_bets_graph(bets_df)
    # ---------------------------↓ TOTAL DATA ↓---------------------------------
    total_amount = go.Scatter(
        x=bets_grouped['date'],
        y=bets_grouped['bet_count'],
        mode='lines+markers',
        marker_size=5,
        line_color='#e8bcb9',
        name='Total Daily Bets Amount',
        hovertemplate='Total Bets Amount: %{y}<br>Date: %{x}'
    )
    total_sum = go.Scatter(
        x=bets_grouped['date'],
        y=bets_grouped['bet_value'],
        mode='lines+markers',
        marker_size=5,
        line_color='#ae445a',
        name='Total Daily Bets Value',
        hovertemplate='Total Bets Value: %{y}<br>Date: %{x}'
    )
    
    total_profit = go.Scatter(
        x=bets_grouped['date'],
        y=bets_grouped['profit'],
        mode='lines+markers',
        marker_size=5,
        line_color = '#f39f5a',
        name='Total Daily Profit',
        hovertemplate='Total Daily Profit: %{y}<br>Date: %{x}'
    )
    
    fig.add_trace(total_amount)
    fig.add_trace(total_sum)
    fig.add_trace(total_profit)
    # ----------------------------↓ MEAN DATA ↓---------------------------------
    mean_amount = go.Scatter(
        x=bets_grouped['date'],
        y=[bets_grouped['bet_count'].mean()] * len(bets_grouped),
        mode='lines',
        line_color='#e8bcb9',
        line_dash='dot',
        name='Mean Bets Amount',
        hovertemplate='Mean Bets Amount: %{y}',
        visible=False,
    )
    mean_sum = go.Scatter(
        x=bets_grouped['date'],
        y=[bets_grouped['bet_value'].mean()] * len(bets_grouped),
        mode='lines',
        line_color='#ae445a',
        line_dash='dot',
        name='Mean Bets Value',
        hovertemplate='Mean Bets Value: %{y}',
        visible=False,
    )
    mean_profit = go.Scatter(
        x=bets_grouped['date'],
        y=[bets_grouped['profit'].mean()] * len(bets_grouped),
        mode='lines',
        line_color = '#f39f5a',
        line_dash='dot',
        name='Mean Profit',
        hovertemplate='Mean Profit: %{y}',
        visible=False,
    )
    
    fig.add_trace(mean_amount)
    fig.add_trace(mean_sum)
    fig.add_trace(mean_profit)
    #------------------------------↓ GAME DATA ↓--------------------------------
    for game_id in sorted(bets_df['bet_game_id'].unique()):
        GAME: str = GAME_NAMES[game_id - 1]
        game_data = prepare_df_for_bets_graph(bets_df, game_id)
        game_amount = go.Scatter(
            x=game_data['date'],
            y=game_data['bet_count'],
            mode='lines+markers',
            marker_size=5,
            line_color='#e8bcb9',
            name=f'Total Daily Bets Amount: {GAME}',
            hovertemplate=create_game_hoverstring(GAME, 'Total Bets Amount'),
            visible=False,
        )
        game_sum = go.Scatter(
            x=game_data['date'],
            y=game_data['bet_value'],
            mode='lines+markers',
            marker_size=5,
            line_color='#ae445a',
            name=f'Total Daily Bets Value: {GAME}',
            hovertemplate=create_game_hoverstring(GAME, 'Total Bets Value'),
            visible=False,
        )
        game_profit = go.Scatter(
            x=game_data['date'],
            y=game_data['profit'],
            mode='lines+markers',
            marker_size=5,
            line_color='#f39f5a',
            name=f'Total Daily Profit: {GAME}',
            hovertemplate=create_game_hoverstring(GAME, 'Total Daily Profit'),
            visible=False,
        )
        game_mean_amount = go.Scatter(
            x=game_data['date'],
            y=[game_data['bet_count'].mean()] * len(game_data),
            mode='lines',
            line_color='#e8bcb9',
            line_dash='dot',
            name=f'Mean Bets Amount: {GAME}',
            hovertemplate=GAME + ' Mean Bets Amount: %{y}',
            visible=False,
        )
        game_mean_sum = go.Scatter(
            x=game_data['date'],
            y=[game_data['bet_value'].mean()] * len(game_data),
            mode='lines',
            line_color='#ae445a',
            line_dash='dot',
            name=f'Mean Bets Value: {GAME}',
            hovertemplate=GAME + ' Mean Bets Value: %{y}',
            visible=False,         
        )
        game_mean_profit = go.Scatter(
            x=game_data['date'],
            y=[game_data['profit'].mean()] * len(game_data),
            mode='lines',
            line_color='#f39f5a',
            line_dash='dot',
            name=f'Mean Profit: {GAME}',
            hovertemplate=GAME + ' Mean Profit: %{y}',
            visible=False,             
        )
        fig.add_trace(game_amount)
        fig.add_trace(game_sum)
        fig.add_trace(game_profit)
        fig.add_trace(game_mean_amount)
        fig.add_trace(game_mean_sum)
        fig.add_trace(game_mean_profit)

    fig.update_layout(
        modebar_remove=[
            'toImage',
            'zoom',
            'pan',
            'zoomin',
            'zoomout',
            'autoscale',
            'select',
            'lasso', 
        ],
        xaxis={
            'rangeselector': {
                'buttons': [
                    {
                        'count': 1,
                        'label': '1m',
                        'step': 'month',
                        'stepmode': 'backward',
                    },
                    {
                        'count': 7,
                        'label': '1w',
                        'step': 'day',
                        'stepmode': 'backward'
                    },
                    {
                        'step': 'all',
                    },
                ],
                'x': 0.21,         
            },
        
        'rangeslider': {
            'visible': True,
            'bgcolor': '#451952',
            'thickness': 0.20,
        },
        'type': 'date',
        },
        updatemenus=[
            {
                'type': 'dropdown',
                'direction': 'down',
                'active': 0,
                'bordercolor': '#e8bcb9',
                'x': 0.18,
                'y': 1.2,
                'buttons': create_plot_buttons()
            },
        ]
    )
    fig.update_xaxes(
        rangeselector_bgcolor='#1d1a39',
        rangeselector_activecolor='#451952',
        rangeselector_borderwidth=1,
        rangeselector_bordercolor='#e8bcb9',
    )
    fig.update_yaxes(zerolinecolor='rgba(0,0,0,0)')
    return fig


In [4]:
import datetime
import random


def create_test_df(bets_per_day: int = 10) -> pd.DataFrame:
    dfs_list = []
    for i in range(bets_per_day):
        DEVIATION_MULTIPLICATOR = 1 
        start_date = datetime.date(2023, 11, 28)
        dates = []
        bet_values = []
        user_winnings = []
        for i in range(100):
            delta = datetime.timedelta(days=i)
            new_date = start_date - delta
            dates.append(new_date)
            # ----------------------------------------------------------------------
            value_multiplicator = random.random() * DEVIATION_MULTIPLICATOR       
            value = round(i * value_multiplicator)
            bet_values.append(value)
            # ----------------------------------------------------------------------
            bet_won = True if (random.random() >= 0.9) else False
            user_win = round(value * 2.5 * (random.random() + 1)) if bet_won else 0
            user_winnings.append(user_win)
        
        dates.reverse()    
        bet_game_ids = random.choices(list(range(1, 11)), k=100)
        df = pd.DataFrame(
            {
                'bet_id': list(range(1, 101)),
                'bet_value': bet_values,
                'user_winnings': user_winnings,
                'bet_game_id': bet_game_ids,
                'date': dates, 
            }
        )
        dfs_list.append(df)
    result_df = pd.concat(dfs_list)
    return result_df


In [5]:
bets_df = create_test_df(bets_per_day=100)
bets_graph = create_bets_graph(bets_df)
bets_graph.show()